In [1]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/FYP')
# !git clone https://github.com/gusye1234/LightGCN-PyTorch.git

In [3]:
import os
from os.path import join
import sys
import yaml
import torch
import numpy as np
import pandas as pd
from time import time
from pprint import pprint
from tqdm import tqdm

# Data

In [4]:
import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/FYP/lightgcn/code')

Benchmark datasets: Gowalla, Yelp2018 and Amazon-book and one small dataset LastFM.

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_name = "gowalla"

In [6]:
from lightgcn.code.dataloader import Loader

Read successful
Read successful
Cpp extension not loaded
loading [./lightgcn/data/yelp2018]
1237259 interactions for training
324147 interactions for testing
===========config================
{'A_n_fold': 100,
 'A_split': False,
 'bigdata': False,
 'bpr_batch_size': 2048,
 'decay': 0.005,
 'dropout': 0,
 'keep_prob': 0.6,
 'latent_dim_rec': 64,
 'lightGCN_n_layers': 3,
 'lr': 0.001,
 'multicore': 1,
 'pretrain': 0,
 'ssl_aug_type': 'ED',
 'ssl_ratio': 0.1,
 'ssl_reg': 0.1,
 'ssl_temp': 0.2,
 'test_u_batch_size': 5}
cores for test: 1
comment: lgn
tensorboard: 1
LOAD: 0
Weight path: ./checkpoints
Test Topks: [1, 3, 9]
using bpr loss
===========end===================
Cpp extension not loaded


In [7]:
import time

In [8]:
from timeit import default_timer


class Timer(object):
    """Timer class.
    `Original code <https://github.com/miguelgfierro/pybase/blob/2298172a13fb4a243754acbc6029a4a2dcf72c20/log_base/timer.py>`_.
    Examples:
        >>> import time
        >>> t = Timer()
        >>> t.start()
        >>> time.sleep(1)
        >>> t.stop()
        >>> t.interval < 1
        True
        >>> with Timer() as t:
        ...   time.sleep(1)
        >>> t.interval < 1
        True
        >>> "Time elapsed {}".format(t) #doctest: +ELLIPSIS
        'Time elapsed 1...'
    """

    def __init__(self):
        self._timer = default_timer
        self._interval = 0
        self.running = False

    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def __str__(self):
        return "{:0.4f}".format(self.interval)

    def start(self):
        """Start the timer."""
        self.init = self._timer()
        self.running = True

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        try:
            self._interval = self.end - self.init
            self.running = False
        except AttributeError:
            raise ValueError(
                "Timer has not been initialized: use start() or the contextual form with Timer() as t:"
            )

    @property
    def interval(self):
        """Get time interval in seconds.
        Returns:
            float: Seconds.
        """
        if self.running:
            raise ValueError("Timer has not been stopped, please use stop().")
        else:
            return self._interval

In [9]:
def load(self, model_path=None):
  """Load an existing model.
  Args:
      model_path: Model path.
  Raises:
      IOError: if the restore operation failed.
  """
  try:
      self.saver.restore(self.sess, model_path)
  except Exception:
      raise IOError(
          "Failed to find any matching files for {0}".format(model_path)
      )

In [ ]:
import world
import utils
from world import cprint
import torch
import numpy as np
from tensorboardX import SummaryWriter
import time
import Procedure
from os.path import join
# ==============================
utils.set_seed(world.seed)
print(">>SEED:", world.seed)
# ==============================
import register
from register import dataset

Recmodel = register.MODELS[world.model_name](world.config, dataset)
Recmodel = Recmodel.to(world.device)
print(Recmodel)
print("Checked")
bpr = utils.BPRLoss(Recmodel, world.config)

weight_file = utils.getFileName()
print(f"load and save to {weight_file}")
if world.LOAD:
    try:
        Recmodel.load_state_dict(torch.load(weight_file,map_location=torch.device('cpu')))
        world.cprint(f"loaded model weights from {weight_file}")
    except FileNotFoundError:
        print(f"{weight_file} not exists, start from beginning")
Neg_k = 1

# init tensorboard
if world.tensorboard:
    w : SummaryWriter = SummaryWriter(
                                    join(world.BOARD_PATH, time.strftime("%m-%d-%Hh%Mm%Ss-") + "-" + world.comment)
                                    )
else:
    w = None
    world.cprint("not enable tensorflowboard")

try:
    best_epoch = 0
    best_result = np.zeros([2], dtype=float)

    for epoch in range(world.TRAIN_epochs):
        train_start = time.time()
        eval_time = 0
        if epoch %10 == 0:
            cprint("[TEST]")
            eval_start = time.time()
            print(Recmodel)
            Procedure.Test(dataset, Recmodel, epoch, w, world.config['multicore'])
            eval_end = time.time()
            eval_time = eval_end - eval_start
        print("Finish test")
        output_information = Procedure.BPR_train_original(dataset, Recmodel, bpr, epoch, neg_k=Neg_k,w=w)
        train_end = time.time()
        train_time = train_end - train_start
        print(f'EPOCH[{epoch+1}/{world.TRAIN_epochs}] (train){train_time:.2f} + (eval){eval_time:.2f} :{output_information}')       
finally:
    if world.tensorboard:
        w.close()

>>SEED: 42
loading [./lightgcn/data/yelp2018]
1237259 interactions for training
324147 interactions for testing
===========config================
{'A_n_fold': 100,
 'A_split': False,
 'bigdata': False,
 'bpr_batch_size': 2048,
 'decay': 0.005,
 'dropout': 0,
 'keep_prob': 0.6,
 'latent_dim_rec': 64,
 'lightGCN_n_layers': 3,
 'lr': 0.001,
 'multicore': 1,
 'pretrain': 0,
 'ssl_aug_type': 'ED',
 'ssl_ratio': 0.1,
 'ssl_reg': 0.1,
 'ssl_temp': 0.2,
 'test_u_batch_size': 5}
cores for test: 1
comment: lgn
tensorboard: 1
LOAD: 0
Weight path: ./checkpoints
Test Topks: [1, 3, 9]
using bpr loss
===========end===================
use NORMAL distribution initilizer
loading adjacency matrix
successfully loaded...
don't split the matrix
lgn is already to go(dropout:0)
LightGCN(
  (embedding_user): Embedding(31668, 64)
  (embedding_item): Embedding(38048, 64)
  (f): Sigmoid()
)
Checked
load and save to ./recommender/checkpoints/lgn-yelp2018-3-64.pth.tar
[TEST]
LightGCN(
  (embedding_user): Embedding(

In [ ]:
print(f"Took {training_time.interval:.2f} seconds for training.")